# 1. 한국 주식 리스팅 (섹터, 거래소 정보 포함)

### investing.com에서 가져온 리스트/섹터 데이터, fdr에서 가져온 리스트 데이터 비교/전처리

In [ ]:
import pandas as pd

kosdaq_inv = pd.read_csv('stock_codes/kosdaq_list_inv.csv')
kospi_inv = pd.read_csv('stock_codes/kospi_list_inv.csv')

In [ ]:
# kosdaq_inv에 거래소 라는 컬럼을 추가하고 거래소 이름을 넣는다.
kosdaq_inv['거래소'] = 'KOSDAQ'
# kospi_inv에 거래소 라는 컬럼을 추가하고 거래소 이름을 넣는다.
kospi_inv['거래소'] = 'KOSPI'
# kosdaq_inv와 kospi_inv를 합친다.
krx_inv = pd.concat([kosdaq_inv, kospi_inv])
krx_inv.reset_index(drop=True, inplace=True)
# krx_list_inv.csv로 저장한다.
krx_inv.to_csv('stock_codes/krx_list_inv.csv', index=False)

In [ ]:
krx_inv = pd.read_csv('stock_codes/krx_list_inv.csv')
# 종목명, 기호, 거래소 컬럼만 남긴다.
krx_inv = krx_inv[['종목명', '기호', '거래소']]
krx_inv

In [ ]:
krx_fdr = pd.read_csv('stock_codes/krx_list_fdr.csv')
# krx_fdr의 Code에서 제일 뒤에서부터 3자리를 지운다.
krx_fdr['Code'] = krx_fdr['Code'].str.slice(stop=-3)
# Name, Code, Market 컬럼만 남긴다.
krx_fdr = krx_fdr[['Name', 'Code', 'Market']]
# Name 컬럼을 '종목명'으로, Code 컬럼을 '기호'로, Market 컬럼을 '거래소'로 바꾼다.
krx_fdr.columns = ['종목명', '기호', '거래소']
krx_fdr

In [ ]:
# '기호'의 중복값을 확인한다.
print(krx_inv['기호'].duplicated().sum())
print(krx_fdr['기호'].duplicated().sum())

# '종목명'의 중복값을 확인한다.
print(krx_inv['종목명'].duplicated().sum())
print(krx_fdr['종목명'].duplicated().sum())

In [ ]:
# krx_inv에서 기호가 중복되는 행을 전부 보여준다.
krx_inv[krx_inv['기호'].duplicated(keep=False)].sort_values('기호')

In [ ]:
# krx_inv에서 '기호'의 중복값을 제거한다.
krx_inv.drop_duplicates(subset='기호', keep='first', inplace=True)

In [ ]:
# krx_inv에서 종목명이 중복되는 행을 전부 보여준다.
krx_inv[krx_inv['종목명'].duplicated(keep=False)].sort_values('종목명')
# -> 종목명은 같지만 종목코드가 다른 경우인데, 종목코드를 각각 검색 했을 때 전부 거래소에 존재하므로 둘 다 살린다.

In [ ]:
# null 값이 있는지 확인한다.
print(krx_inv.isnull().sum())
print(krx_fdr.isnull().sum())

In [ ]:
# 거래소 값의 종류를 확인한다.
print(krx_inv['거래소'].unique())
print(krx_fdr['거래소'].unique())

In [ ]:
# krx_fdr의 '거래소' 컬럼의 KOSDAQ GLOBAL을 KOSDAQ으로 바꾼다.
krx_fdr['거래소'] = krx_fdr['거래소'].replace('KOSDAQ GLOBAL', 'KOSDAQ')

In [ ]:
# krx_inv와 krx_fdr 비교
print(krx_inv.shape)
print(krx_fdr.shape)

In [ ]:
# krx_inv의 기호와 krx_fdr의 기호를 비교.
# krx_inv에만 있는 것.
only_krx_inv = krx_inv[~krx_inv['기호'].isin(krx_fdr['기호'])]
print(only_krx_inv)

In [ ]:
# krx_inv에만 있는 것을 krx_fdr에 추가한다.
krx_fdr = pd.concat([krx_fdr, only_krx_inv])

In [ ]:
# krx_fdr의 기호와 krx_inv의 기호를 비교.
# krx_fdr에만 있는 것.
only_krx_fdr = krx_fdr[~krx_fdr['기호'].isin(krx_inv['기호'])]
print(only_krx_fdr)
# -> 일반주(보통주)와 우선주의 문제인데, 일단 우선주도 다 살린다.

In [ ]:
# only_krx_fdr을 전부 krx_inv에 추가한다.
krx_inv = pd.concat([krx_inv, only_krx_fdr])

In [ ]:
# 다시 krx_inv와 krx_fdr 비교하여 서로 열 개수가 같음을 확인.
print(krx_inv.shape)
print(krx_fdr.shape)

In [ ]:
# 두 데이터프레임을 기호를 기준으로 합친다.
kr = pd.merge(krx_inv, krx_fdr, on='기호', how='outer')
kr

In [ ]:
# 종목명_x와 종목명_y가 서로 다른 것을 확인.
kr[kr['종목명_x'] != kr['종목명_y']]
# -> 단순 영한 번역의 차이로 생긴 문제이다.

In [ ]:
kr.columns = ['종목명', '기호', '거래소', '종목명2', '거래소2']
kr.drop(['종목명2', '거래소2'], axis=1, inplace=True)
# 종목명을 기준으로 정렬한다.
kr.sort_values('종목명', inplace=True)
kr.reset_index(drop=True, inplace=True)
kr

In [ ]:
# 섹터 데이터 불러오기
kosdaq_sector = pd.read_csv('stock_codes/kosdaq_sector.csv')
kospi_sector = pd.read_csv('stock_codes/kospi_sector.csv')

In [ ]:
# 섹터 데이터 중복 데이터 확인
print(kosdaq_sector['기호'].duplicated().sum())
print(kospi_sector['기호'].duplicated().sum())

In [ ]:
# 중복 데이터를 불러온다.
print(kosdaq_sector[kosdaq_sector['기호'].duplicated(keep=False)].sort_values('기호'))
print(kospi_sector[kospi_sector['기호'].duplicated(keep=False)].sort_values('기호'))

In [ ]:
# 중복 데이터 제거
kosdaq_sector.drop_duplicates(subset='기호', keep='first', inplace=True)
kospi_sector.drop_duplicates(subset='기호', keep='first', inplace=True)

In [ ]:
# kosdaq_sector와 kospi_sector를 합친다.
kr_sector = pd.concat([kosdaq_sector, kospi_sector])
kr_sector.reset_index(drop=True, inplace=True)
# 종목명, 기호, 섹터 컬럼만 남긴다.
kr_sector = kr_sector[['종목명', '기호', '섹터']]
kr_sector

In [ ]:
# kr과 kr_sector 비교.
print(kr.shape)
print(kr_sector.shape)

In [ ]:
# kr의 기호와 kr_sector의 기호를 비교.
# kr에만 있는 것.
only_kr = kr[~kr['기호'].isin(kr_sector['기호'])]
only_kr

In [ ]:
# kr_sector에만 있는 것.
only_kr_sector = kr_sector[~kr_sector['기호'].isin(kr['기호'])]
only_kr_sector

In [ ]:
# kr에만 있는 값(only_kr)을 제거한다.
kr = kr.drop(only_kr.index)
kr

In [ ]:
# kr과 kr_sector을 합친다.
kr = pd.merge(kr, kr_sector, on='기호', how='outer')
kr

In [ ]:
kr.columns = ['종목명', '기호', '거래소', '종목명2', '섹터']
kr.drop(['종목명2'], axis=1, inplace=True)
kr

# 2. 미국 주식 리스팅 (섹터, 거래소 정보 포함)

### investing.com에서 가져온 리스트/섹터 데이터 비교/전처리

In [ ]:
nasdaq_inv = pd.read_csv('stock_codes/nasdaq_list_inv.csv')
nyse_inv = pd.read_csv('stock_codes/nyse_list_inv.csv')
nasdaq_sector = pd.read_csv('stock_codes/nasdaq_sector.csv')
nyse_sector = pd.read_csv('stock_codes/nyse_sector.csv')

In [ ]:
# 중복 데이터 확인
print(nasdaq_inv['기호'].duplicated().sum())
print(nyse_inv['기호'].duplicated().sum())
print(nasdaq_sector['기호'].duplicated().sum())
print(nyse_sector['기호'].duplicated().sum())

In [ ]:
# 중복 데이터를 불러온다.
print(nasdaq_inv[nasdaq_inv['기호'].duplicated(keep=False)].sort_values('기호'))
print(nyse_inv[nyse_inv['기호'].duplicated(keep=False)].sort_values('기호'))
print(nasdaq_sector[nasdaq_sector['기호'].duplicated(keep=False)].sort_values('기호'))
print(nyse_sector[nyse_sector['기호'].duplicated(keep=False)].sort_values('기호'))
# -> 모든 값이 같기 때문에 이 중복 데이터들은 제거한다.

In [ ]:
# 중복 데이터 제거
nasdaq_inv.drop_duplicates(subset='기호', keep='first', inplace=True)
nyse_inv.drop_duplicates(subset='기호', keep='first', inplace=True)
nasdaq_sector.drop_duplicates(subset='기호', keep='first', inplace=True)
nyse_sector.drop_duplicates(subset='기호', keep='first', inplace=True)

In [ ]:
# nasdaq_inv와 nasdaq_sector의 기호를 비교한다.
print(nasdaq_inv.shape)
print(nasdaq_sector.shape)

In [ ]:
# nasdaq_inv에만 있는 것.
only_nasdaq_inv = nasdaq_inv[~nasdaq_inv['기호'].isin(nasdaq_sector['기호'])]
only_nasdaq_inv

In [ ]:
# nasdaq_sector에만 있는 것.
only_nasdaq_sector = nasdaq_sector[~nasdaq_sector['기호'].isin(nasdaq_inv['기호'])]
only_nasdaq_sector

In [ ]:
# nasdaq_inv에만 있는 값(ony_nasdaq_inv)을 제거한다.
nasdaq_inv = nasdaq_inv.drop(only_nasdaq_inv.index)
nasdaq_inv

In [ ]:
# nasdaq_inv에 거래소 컬럼을 추가하고 거래소 이름을 넣는다.
nasdaq_inv['거래소'] = 'NASDAQ'
# nasdaq_inv에서 '종목명'과 '기호', '거래소' 컬럼만 남긴다.
nasdaq_inv = nasdaq_inv[['종목명', '기호', '거래소']]
# nasdaq_sector에서 '종목명'과 '기호', '섹터' 컬럼만 남긴다.
nasdaq_sector = nasdaq_sector[['종목명', '기호', '섹터']]
# nasdaq_sector와 nasdaq_inv를 기호를 기준으로 합친다.
nasdaq = pd.merge(nasdaq_inv, nasdaq_sector, on='기호', how='outer')
nasdaq

In [ ]:
nasdaq.columns = ['종목명', '기호', '거래소', '종목명2', '섹터']
nasdaq.drop(['종목명2'], axis=1, inplace=True)
# 기호 기준으로 정렬한다.
nasdaq.sort_values('기호', inplace=True)
nasdaq.reset_index(drop=True, inplace=True)
nasdaq

In [ ]:
# nyse_inv와 nyse_sector의 기호를 비교한다.
print(nyse_inv.shape)
print(nyse_sector.shape)

In [ ]:
# nyse_inv에만 있는 것.
only_nyse_inv = nyse_inv[~nyse_inv['기호'].isin(nyse_sector['기호'])]
print(only_nyse_inv.shape)
only_nyse_inv

In [ ]:
# nyse_sector에만 있는 것.
only_nyse_sector = nyse_sector[~nyse_sector['기호'].isin(nyse_inv['기호'])]
only_nyse_sector

In [ ]:
# nyse_inv에만 있는 값(ony_nyse_inv)을 제거한다.
nyse_inv = nyse_inv.drop(only_nyse_inv.index)
nyse_inv

In [ ]:
# nyse_inv에 거래소 컬럼을 추가하고 거래소 이름을 넣는다.
nyse_inv['거래소'] = 'NYSE'
# nyse_inv에서 '종목명'과 '기호', '거래소' 컬럼만 남긴다.
nyse_inv = nyse_inv[['종목명', '기호', '거래소']]
# nyse_sector에서 '종목명'과 '기호', '섹터' 컬럼만 남긴다.
nyse_sector = nyse_sector[['종목명', '기호', '섹터']]
# nyse_sector와 nyse_inv를 기호를 기준으로 합친다.
nyse = pd.merge(nyse_inv, nyse_sector, on='기호', how='outer')
nyse

In [ ]:
nyse.columns = ['종목명', '기호', '거래소', '종목명2', '섹터']
nyse.drop(['종목명2'], axis=1, inplace=True)
# 기호 기준으로 정렬한다.
nyse.sort_values('기호', inplace=True)
nyse.reset_index(drop=True, inplace=True)
nyse

In [ ]:
# nasdaq, nyse 데이터프레임을 합친다.
us = pd.concat([nasdaq, nyse])
us

# 3. CSV 저장

In [ ]:
# kr, us 데이터프레임을 csv로 저장한다.
kr.to_csv('stock_codes/kr_list_final.csv', index=False)
us.to_csv('stock_codes/us_list_final.csv', index=False)